In [16]:
from pathlib import Path
from collections import defaultdict
import warnings
import math
import re
import time

import soundfile as sf
import numpy as np
import pykakasi
from lhotse import RecordingSet, Recording, SupervisionSegment, SupervisionSet, CutSet
from lhotse.recipes.utils import read_manifests_if_cached
from phonemizer import phonemize
from phonemizer.backend import EspeakBackend
from phonemizer.separator import Separator

In [107]:
ipa_backend = EspeakBackend('ja')
ipa_separator = Separator(phone=None, word=None)
src_dir = Path("/home/shahn/Documents/icefall_github/egs/reazonspeech/ASR/data/manifests")

dataset_parts = (
    # "train",
    # "dev",
    "test",
)

prefix = "reazonspeech"
suffix = "jsonl.gz"
print(f"Read manifests...")
manifests = read_manifests_if_cached(
    dataset_parts=dataset_parts,
    output_dir=src_dir,
    prefix=f"{prefix}_",
    suffix=suffix,
)

Read manifests...


In [108]:
hiragana = set(['ゖ', 'ぉ', 'ぅ', 'ゔ', 'ぬ', 'ゆ', 'ど', 'ー', 'ぐ', 'げ', 'づ', 'お', 'ち', 'ぜ', 'ぺ', 'ぁ', 'ょ', 'こ', 'み', 'ろ', 'ぇ', 'び', 'だ', 'と', 'ほ', 'え', 'れ', 'じ', 'ぼ', 'た', 'ゃ', 'よ', 'ぎ', 'い', 'け', 'て', 'ふ', 'や', 'べ', 'す', 'り', 'は', 'ご', 'ね', 'し', 'も', 'ぱ', 'へ', 'め', 'ん', 'に', 'ぶ', 'む', 'で', 'う', 'ぴ', 'ひ', 'な', 'ゅ', 'ら', 'つ', 'る', 'か', 'せ', 'く', 'あ', 'の', 'が', 'さ', 'わ', 'ざ', 'ま', 'ぃ', 'ず', 'そ', 'ぞ', 'ぷ', 'ば', 'ぽ', 'を', 'き', 'っ'])
# katakana = set(['ー', 'ピ', 'ヅ', 'キ', 'ウ', 'タ', 'グ', 'ヲ', 'ツ', 'ヨ', 'ニ', 'ソ', 'イ', 'バ', 'テ', 'パ', 'ギ', 'デ', 'ジ', 'ラ', 'ッ', 'セ', 'ク', 'ェ', 'ア', 'モ', 'ワ', 'ゴ', 'ァ', 'ガ', 'ゼ', 'シ', 'ヘ', 'ロ', 'ミ', 'ョ', 'カ', 'コ', 'サ', 'チ', 'ヤ', 'ト', 'ム', 'ゲ', 'ド', 'ビ', 'ス', 'ザ', 'ン', 'ダ', 'エ', 'ネ', 'メ', 'ブ', 'オ', 'ナ', 'フ', 'ゾ', 'ィ', 'ペ', 'レ', 'プ', 'ユ', 'ボ', 'マ', 'ケ', 'ル', 'ベ', 'ヒ', 'ズ', 'ノ', 'ハ', 'ャ', 'ュ', 'リ', 'ホ', 'ポ'])
# roman = set("abcdefghijklmnopqrstuvwxyz")
# len(hiragana), len(katakana), len(roman)

In [109]:
normalization_rules = [
    ["きゃ", "캬"], ["きゅ", "큐"], ["きょ", "쿄"],
    ["しゃ", "샤"], ["しゅ", "슈"], ["しょ", "쇼"],
    ["ちゃ", "챠"], ["ちゅ", "츄"], ["ちょ", "쵸"],
    ["にゃ", "냐"], ["にゅ", "뉴"], ["にょ", "뇨"],
    ["ひゃ", "햐"], ["ひゅ", "휴"], ["ひょ", "효"],
    ["ぎゃ", "먀"], ["みゅ", "뮤"], ["みょ", "묘"],
    ["りゃ", "랴"], ["りゅ", "류"], ["りょ", "료"],
    ["みゃ", "갸"], ["ぎゅ", "규"], ["じょ", "교"],
    ["じゃ", "자"], ["じゅ", "주"], ["ぎょ", "조"],
    ["ぢゃ", "자"], ["ぢゅ", "주"], ["ぢょ", "조"],
    ["びゃ", "뱌"], ["びゅ", "뷰"], ["びょ", "뵤"],
    ["ぴゃ", "퍄"], ["ぴゅ", "퓨"], ["ぴょ", "표"],
    ["い", "이"], ["ゐ", "이"],
    ["え", "에"], ["ゑ", "에"],
    ["お", "오"], ["を", "오"],
    ["じ", "지"], ["ぢ", "지"],
    ["ず", "주"], ["づ", "주"],
]
hiragana.update(set([x[1] for x in normalization_rules]))

In [104]:
manifests["test"]

{'recordings': RecordingSet(len=100), 'supervisions': SupervisionSet(len=100)}

In [112]:
# vocab = set()
num_processed = 0
num_filtered = 0

with warnings.catch_warnings():
    for part in dataset_parts:
        supervisions = manifests[part]['supervisions']
        warnings.simplefilter("ignore")
        kakasi = pykakasi.kakasi()
        kakasi.setMode("K", "H")
        kakasi.setMode("H", None)
        kakasi.setMode("J", "H")
        converter = kakasi.getConverter()
    
        for idx, sup in enumerate(supervisions):
            text = re.sub("[［］〈〉・♪《》−．⁉“”◆）‼≪]", " ", sup.text)
            output = converter.do(text)
            for rule in normalization_rules:
                output = output.replace(rule[0], rule[1])
            filtered = False
            for word in output:
                # vocab.add(word)
                if word not in hiragana:
                    # num_filtered += 1
                    # filtered = True
                    # break
                    if not filtered:
                        print(f"[{word}", end="")
                        filtered = True
                    else:
                        print(f", {word}", end="")
                else:
                    vocab.add(word)
            if filtered:
                print(f"], {text}")
            # if not filtered:
            #     num_processed += 1
# print(num_processed, num_filtered)
# print(vocab)

[m], 六十mということはえっ
[ ,  ], ちょっと変わった宗教団体がやってるご当地ハンバーガー高級なハンバーガーというような認識だと思うんですけれどもこれムン ソンミョン教祖とハン ハクチャ総裁が統一運動にまい進する中で非常にお昼時間に時間を取っている時間がないということでハンバーガーで済ませていたというそういうところから発祥なんですよね
[N, A, T, O,  ], 中国の習近平国家主席とロシアのプーチン大統領はきのう北京で会談しNATO 北大西洋条約機構のさらなる拡大に反対するとした共同声明を発表しました
[ ,  ],  もう一度ご覧になりたい方はぜひチェックしてください さあ
[ ], 東京 表参道のアクセサリー店で起きた強盗事件で被害品の転売役とみられる四十四歳の男が新たに逮捕されました
[L, I, N, E], こんな母からのLINEはイヤだどんなの
[G, O, V], GOVというのはガバナー知事の略称なのでトップの知事要は最高の知事と自画自賛した動画といえるんですね
[ ,  ],  山の中で地図とコンパスを頼りに目的地を目指したり蓼科山の登山さらには肝試しも楽しまれました 
[ ,  ],  現在は関東大震災の身元不明の遺骨を祭る東京都慰霊堂が立っている 
[ ,  ],  五人の子供たちそれぞれがストレスなく勉強できるスペースが 
[ ], これもタコ物なんですけどカプト メデューサといってタコ物の中で一番王様っていわれる大型の種類ですね
[ ,  ,  ], 情熱的なフラメンコの街ヘレス デ ラ フロンテーラ
[S, t, a, n, d, b, y, W, o, m, e, n], 女性議員を支援する団体StandbyWomenの濱田代表は女性議員は男性議員に比べて地元出身の割合が低いため支持基盤が弱く被害に遭っても票を失う恐れがあると思ってなかなか声を上げにくいといいます
[ , k, m], その柳通りを進んでしまうともう残りはおよそ一 五kmということになります
[ ,  ],  今日は春キャベツの食感を生かした豚平焼きとやわらかい春のかぶで上品なさっと煮を作ります 
[ , ％], 今日発表された六月の都市部の若者の失業率も二十一 三％と雇用状況が悪化しています
[ ], だがその経験こそが今の俳優 斎藤工を作り上げた

In [97]:
len(supervisions)

618004

In [9]:
sup = manifests['test']['supervisions']
text= sup[0].text
print(text)
print(phonemize(text, language='ja', backend='espeak', preserve_punctuation=True, njobs=1))
print(phonemize(text, language='ja', backend='espeak', preserve_punctuation=False, njobs=1))

ライブ映像です菅総理のコメントがこれから発表されます
ɽä i ɸɯᵝ (en)dʒapəniːz(ja)le̞tə (en)tʃaɪniːz(ja)le̞tə (en)tʃaɪniːz(ja)le̞tə te̞ (en)dʒapəniːz(ja)le̞tə sɯᵝ (en)tʃaɪniːz(ja)le̞tə (en)tʃaɪniːz(ja)le̞tə (en)tʃaɪniːz(ja)le̞tə no̞ ko̞ me̞ ɴ to̞ kä (en)dʒapəniːz(ja)le̞tə ko̞ ɽe̞ kä ɽä (en)tʃaɪniːz(ja)le̞tə (en)tʃaɪniːz(ja)le̞tə sä ɽe̞ mä sɯᵝ 
ɽä i ɸɯᵝ (en)dʒapəniːz(ja)le̞tə (en)tʃaɪniːz(ja)le̞tə (en)tʃaɪniːz(ja)le̞tə te̞ (en)dʒapəniːz(ja)le̞tə sɯᵝ (en)tʃaɪniːz(ja)le̞tə (en)tʃaɪniːz(ja)le̞tə (en)tʃaɪniːz(ja)le̞tə no̞ ko̞ me̞ ɴ to̞ kä (en)dʒapəniːz(ja)le̞tə ko̞ ɽe̞ kä ɽä (en)tʃaɪniːz(ja)le̞tə (en)tʃaɪniːz(ja)le̞tə sä ɽe̞ mä sɯᵝ 


In [61]:
text = "ゆうゆう, ゆーゆー"
text = "ふぁしりてぃをつうじてかん"
print(phonemize(text, language='ja', backend='espeak', preserve_punctuation=True, njobs=1))
print(phonemize(text, language='ja', backend='espeak', preserve_punctuation=False, njobs=1))

ɸɯᵝ (en)dʒapəniːz(ja)le̞tə ɕi ɽi te̞ (en)dʒapəniːz(ja)le̞tə o̞ tsɯᵝ ɯᵝ ɕi (en)dʒapəniːz(ja)le̞tə te̞ kä ɴ 
ɸɯᵝ (en)dʒapəniːz(ja)le̞tə ɕi ɽi te̞ (en)dʒapəniːz(ja)le̞tə o̞ tsɯᵝ ɯᵝ ɕi (en)dʒapəniːz(ja)le̞tə te̞ kä ɴ 


In [63]:
'ぁ' == 'あ'

False

In [62]:
!espeak-ng -q -x --ipa -v ja "ふぁしり"

ɸˈɯᵝ (en)dʒˈapəniːz(ja)lˈe̞tə ɕˈi ɽˈi


In [15]:
import warnings
import pykakasi

kakasi = pykakasi.kakasi()

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    kakasi.setMode("K", "H")
    kakasi.setMode("J","H")
    conv = kakasi.getConverter()
    text = "ライブ映像です菅総理のコメントがこれから発表されます"
    result = conv.do(text)
print(result)

らいぶえいぞうですかんそうりのこめんとがこれからはっぴょうされます


In [116]:
!espeak-ng -q -x --ipa -v es "haya enyeso jet gin"

ˈajja eɲˈeso xˈet xˈin
